In [1]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,MaxPooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os

In [2]:
img = cv2.imread("data/close/s0001_00077_0_0_0_0_0_01.png")
img.shape

(100, 100, 3)

In [3]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),  #################
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#32 convolution filters used each of size 3x3
#again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),

#64 convolution filters used each of size 3x3
#choose the best features via pooling
    
#randomly turn neurons on and off to improve convergence
    Dropout(0.25),
#flatten since too many dimensions, we only want a classification output
    Flatten(),
#fully connected to get all relevant data
    Dense(128, activation='relu'),
#one more dropout for convergence' sake :) 
    Dropout(0.5),
#output a softmax to squash the matrix into output probabilities
    Dense(2, activation='softmax')
])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 18, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 18, 18, 64)        0

In [4]:
model.compile(optimizer="adam",loss='categorical_crossentropy',metrics=["accuracy"])

# loss='categorical_crossentropy'
# loss="binary_crossentropy"

In [5]:
image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

targetsize = (24,24)                                     ###########################
batchsize = 32

#train_image_gen = img_gen.flow_from_directory("eyes",target_size=(100,100),batch_size=10)

train_dataset = image_generator.flow_from_directory('my_data',batch_size=batchsize,shuffle = True,
                    color_mode='grayscale',target_size= targetsize,subset = 'training',class_mode = 'categorical')

valid_dataset = image_generator.flow_from_directory('my_data',batch_size=batchsize,shuffle = True,
                color_mode='grayscale',target_size= targetsize,subset = 'validation',class_mode = 'categorical')

Found 67919 images belonging to 2 classes.
Found 16979 images belonging to 2 classes.


In [6]:
SPE = len(train_dataset.classes)//batchsize
VS = len(valid_dataset.classes)//batchsize

SPE,VS
# print(train_image_gen.class_indices)

(2122, 530)

In [7]:
#results = model.fit_generator(train_image_gen,epochs=20,steps_per_epoch=243)

model.fit(train_dataset,validation_data= valid_dataset,epochs = 10,steps_per_epoch=SPE,validation_steps = VS)

Epoch 1/10
2122/2122 [==============================] - 939s 442ms/step - loss: 0.2217 - accuracy: 0.9116 - val_loss: 0.2132 - val_accuracy: 0.9129
Epoch 2/10
2122/2122 [==============================] - 136s 64ms/step - loss: 0.0760 - accuracy: 0.9731 - val_loss: 0.1665 - val_accuracy: 0.9353
Epoch 3/10
2122/2122 [==============================] - 140s 66ms/step - loss: 0.0543 - accuracy: 0.9812 - val_loss: 0.1760 - val_accuracy: 0.9259
Epoch 4/10
2122/2122 [==============================] - 148s 70ms/step - loss: 0.0444 - accuracy: 0.9846 - val_loss: 0.2651 - val_accuracy: 0.9206
Epoch 5/10
2122/2122 [==============================] - 145s 69ms/step - loss: 0.0411 - accuracy: 0.9859 - val_loss: 0.3713 - val_accuracy: 0.9017
Epoch 6/10
2122/2122 [==============================] - 162s 76ms/step - loss: 0.0326 - accuracy: 0.9881 - val_loss: 0.2524 - val_accuracy: 0.9232
Epoch 7/10
2122/2122 [==============================] - 151s 71ms/step - loss: 0.0319 - accuracy: 0.9887 - val_loss: 

In [8]:
model.save('models/trained_model_24_2.h5',overwrite = True)